In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import time
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import regex as re


import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from spacy.tokenizer import Tokenizer
from spacy.pipeline import EntityRuler
import re
from spacy.tokens import Span
from spacy.util import filter_spans


In [ ]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
tokenizer = nlp.tokenizer
all_stopwords = nlp.Defaults.stop_words


In [ ]:
#dataframe of descriptions
df = pd.read_csv('data.csv')

In [ ]:
#cleaning the descriptions
def clean_notes(data, col_name):
    data[col_name] = data[col_name].str.replace('lc', 'lamp column')
    data[col_name] = data[col_name].str.replace('l/c', 'lamp column')
    data[col_name] = data[col_name].str.replace('sugg', 'suggested')
    data[col_name] = data[col_name].str.replace('rreturn', 'return')
    data[col_name] = data[col_name].str.replace('o/s', 'outside')

    return data

In [ ]:
df =clean_notes(df, 'job_notes')

In [ ]:
def regex_checker(text, pattern):
    matches = re.finditer(pattern, text)
    for match in matches:
        print (match)


In [ ]:
#initial pattern to match addresses
patterns =  [{"label": "ADDRESS", "pattern": [" ?outside \d+ ?", " ?\d{1,3} [A-Za-z]+ road\.?",
                                             " ?\d{1,3} [A-Za-z]+ rd\.?"," ?\d{1,3} [A-Za-z]+ hill\.?",
                                              " ?\d{1,3} [A-Za-z]+ Hill\.?", " ?\d{1,3} st\.? [A-Za-z]+",
                                              " ?\d{1,3} saint\.? [A-Za-z]+"]}]
                                              

In [ ]:
docs = []
entity_text = []
#list of new entities
new_ents = []

for i in range(len(df)):
    docs.append(df.iloc[i, 3])
    doc = nlp(df.iloc[i, 3])

    #saving the original entities in the description
    original_ents = list(doc.ents)

    #Identifying new multiple-word entities using regex patterns
    for i in range(len(patterns)):
      # because each label can have multiple patterns in a list
        for j in range(len(patterns[i]['pattern'])):
            for match in re.finditer(patterns[i]['pattern'][j], doc.text):
                print (match)
                start, end = match.span()
                span = doc.char_span(start, end)
                if span is not None:
                    #appending start char, end char, and text of entity
                    new_ents.append((span.start, span.end, span.text))
                
        for ent in new_ents:
            start, end, name = ent
            per_ent = Span(doc, start, end, label=patterns[i]['label'])
            original_ents.append(per_ent)    

    #this code prioritizes longer spans over shorter ones to prevent overlap between entities       
    filtered = filter_spans(original_ents)
    doc.ents = filtered
    entity_text.append(doc.ents)

<re.Match object; span=(11, 27), match=' 63 putney hill.'>
<re.Match object; span=(11, 23), match=' outside 95 '>
<re.Match object; span=(19, 30), match=' 95 st john'>
<re.Match object; span=(14, 26), match=' outside 64 '>
<re.Match object; span=(22, 37), match=' 64 lyford road'>
<re.Match object; span=(11, 29), match=' 9 coalecroft road'>
<re.Match object; span=(11, 27), match=' 11 lynwood road'>
<re.Match object; span=(227, 239), match=' outside 76 '>
<re.Match object; span=(210, 222), match=' 12 st johns'>
<re.Match object; span=(13, 24), match=' outside 76'>
<re.Match object; span=(11, 29), match=' 21 charlwood road'>
<re.Match object; span=(11, 27), match=' 26 mitcham road'>
<re.Match object; span=(14, 26), match=' outside 30 '>
<re.Match object; span=(11, 26), match=' 9 beeches road'>
<re.Match object; span=(11, 24), match=' 6 lombard rd'>
<re.Match object; span=(11, 28), match=' 20 queenstown rd'>
<re.Match object; span=(13, 25), match=' outside 15 '>
<re.Match object; span=(21,

In [ ]:
entity_df = pd.DataFrame(list(zip(docs, entity_text)),
               columns =['Text', 'Entity,Label'])


In [ ]:
entity_df.to_csv('entity.csv')